In [ ]:
import numpy as np
import pandas as pd
import scipy.signal as signal
import pywt #信号处理库
import os
from predict_funcs import *
from keras.models import load_model

In [ ]:
#模型的位置
modelpath = 'model/model.h5'

In [ ]:
#查找串口信息
import serial
import serial.tools.list_ports
port_list = list(serial.tools.list_ports.comports())
if len(port_list) == 0:
    print('找不到串口')
else:
    for i in range(0,len(port_list)):
        print(port_list[i])

In [ ]:
#初始化overlay
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay('base.bit')
print('finish')

In [ ]:
#建立TGAM与PYNQ板的蓝牙通信
ser = serial.Serial('/dev/ttyACM1',57600)

In [ ]:
#进行脑波序列的采集、预测，并通过预测结果来控制机械臂
while True:
    rawdata = ser.read(20000).hex() #从TGAM模块获取到脑电序列
    res = get_res(modelpath ,rawdata) #通过get_res函数直接获取到最终的预测结果
    print(res)

    #建立串口通信，并将最终的预测结果送出
    lib = MicroblazeLibrary(base.RPI, ['uart'])
    device = lib.uart_open(14,15)
    
    if res == 1:
        list2 = [0x01]
        print("抓取")
    elif res == 0:
        list2 = [0x02]
        print("放开")
    elif res == 2:
        list2 = [0x03]
        print("上抬")
    elif res == 3:
        list2 = [0x04]
        print("下放")
    elif res == 6:
        list2 = [0x05]
        print("左转")
    elif res == 5:
        list2 = [0x06]
        print("右转")
    lib.uart_write(device,list2,len(list2))